## Konteneryzacja aplikacji Pythonowych

### Obraz bazowy
Najprostszym sposobem jest użycie jednego z istniejących obrazów Dockerowych, dostępnych na [hub.docker.com](hub.docker.com). Który zatem wybrać? Najlepszą praktyką jest minimalny obraz, mający w sobie tylko to, co potrzebne naszej aplikacji. W wielu przypadkach nie opłaca sięe wybierać obrazu, zawierającego w sobie pełen system operacyjny, gdyż jedynie zwiększamy powierzchnię potencjalnego ataku i spowalniamy start kontenera.

Dobrym domyślnym wyborem powinien być więc obraz z kategorii "slim", który w przypadku Debiana jest kilkukrotnie mniejszy - 146MB vs 1.01GB! 

**UWAGA** Nie zalecam stosowania obrazów opartych o Linux Alpine - ponieważ Alpine używa `musl` zamiast `glibc`, nie wszystkie biblioteki Pythonowe wspierają ten system. Ich instalacja może się więc wiązać z koniecznością doinstalowania całego toolchaina opartego o glibc, co zupełnie zniweluje niewielkie zyski w rozmiarze obrazu. W najgorszym razie o problemach z kompatybilnością można przekonać się w czasie działania programu.

### Przypinanie wersji
Zazwyczaj nie wystarczy jedynie wybrać odpowiedni tag wskazujący wersję Pythona- np. `python:3.11`, czy nawet system operacyjny jak `python:3.11.4-slim-bookworm`. W rejestrach dockerowych tagi nie są stałe, więc ten sam tag w teorii może być przypisany do dwóch różniących się wersji - wówczas najlepszy sposób by uniknąć nieprzyjemnych niespodzianek to przypinanie konkretnej wersji:

```
FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5
WORKDIR /usr/app
COPY . .
RUN pip install -r requirements.txt
CMD [ "python", "app.py" ]
```

### Instalacja zależności
Każdy, kto kiedykolwiek budował obraz dockerowy lokalnie lub na CI, w ramach wielokrotnie powtarzającego się procesu wie, jak ważne jest efektywne używanie mechanizmu cache'owania warstw. W przypadku obrazu dla aplikacji Pythonowych, dobrą praktyką jest wczesne wkopiowanie listy zależności - pliku `requirements.txt` - zmienia się on zdecydowanie rzadziej niż kod aplikacji, więc ten ruch zaoszczędzi nam sporo czasu potrzebnego na instalację zależności.
```
FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5
WORKDIR /usr/app

COPY requirements.txt .
RUN pip install -r requirements.txt

COPY . .
CMD [ "python", "app.py" ]
```

#### Trudniejszy przypadek
Kiedy zależnością naszej aplikacji jest biblioteka, która nie jest spaczkowana jako `bdist_wheel`, może być konieczne zbudowanie jej ze źródeł. Wówczas nasz obraz musi zawierać również kompilator i inne narzędzia budowania, które zajmują sporo miejsca, są potencjalnymi wektorami ataku, a zupełnie nie przydadzą się w czasie działania aplikacji. Dobrym wyjściem z problemu jest użycie wieloetapowego budowania obrazu (*multistage build*)

```
FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5 as build
RUN apt-get update
RUN apt-get install -y --no-install-recommends \
	      build-essential gcc

WORKDIR /usr/app
COPY requirements.txt .
RUN python -m venv env
ENV PATH=/usr/app/env/bin:$PATH
RUN pip install -r requirements.txt

FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5
WORKDIR /usr/app
COPY --from=build /usr/app/env ./env
ENV PATH=/usr/app/env/bin:$PATH

COPY . .
CMD [ "python", "app.py" ]
```
Jak widać, w tym przypadku posłużylismy się tu modułem `venv` i zainstalowaliśmy zależności do virtualenva - pomaga nam to uniknąć problemu z przenoszeniem zależności zainstalowanych globalnie, które potencjalnie mogą chcieć wówczas zainstalować część swoich plików w różnych miejscach systemu, o których moglibyśmy nie wiedzieć.
W przypadku tak skonstruowanego obrazu, zaleca się używanie Docker buildkit, który w lepszy sposób operuje cache'm (zwłaszcza z opcją ` --build-arg BUILDKIT_INLINE_CACHE=1`)

### Ograniczenie uprawnień
Dobrą praktyką jest by unikać uruchamiania procesów w kontenerze na koncie roota - wówczas nawet w przypadku "ucieczki" z kontenera lokalny użytkownik w systemie hosta na którego mapował się użytkownik wewnątrz kontenera ma mniejsze uprawnienia i nie powinien móc zrobić zbyt wielu szkód. Nasz Dockerfile po tej zmianie wygląda więc następująco:
```
FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5 as build
RUN apt-get update
RUN apt-get install -y --no-install-recommends \
	      build-essential gcc

WORKDIR /usr/app
COPY requirements.txt .
RUN python -m venv env
ENV PATH=/usr/app/env/bin:$PATH
RUN pip install -r requirements.txt

FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5
RUN groupadd -g 1001 python && \
    useradd -r -u 1001 -g python python
USER 1001

WORKDIR /usr/app
COPY --from=build /usr/app/env ./env
ENV PATH=/usr/app/env/bin:$PATH

COPY . .
CMD [ "python", "app.py" ]
```

### tini
W przypadku, gdy nasza aplikacja odpala wewnątrz więcej procesów (np. z użyciem modułu `multiprocessing` lub `subprocess`), możliwe jest, że część stworzonych procesów zostanie `zombie`. Normalnie jest odpowiedzialnością procesu `init` odziedziczyć procesy, których `parent` się zakończył i zniszczyć wpis w tabeli procesów jądra linuxa, unikając tym samym wycieku. W obrazach pythonowych dostępnych na [hub.docker.com](hub.docker.com) entrypointem jest jednak proces `python`, który nie wypełnia kontraktu wymaganego od `PID 1`. Nie ma on również dobrych domyślnych implementacji handlerów sygnałów, przez co nasza aplikacja nie zareaguje poprawnie np. na `SIGTERM`, utrudniając i spowalniając zatrzymanie kontenera. Aby temu zaradzić możemy się posłużyc małym narzędziem, które jest minimalną implementacją procesu spełniającego odpowiedzialności `PID 1`- `tiny`. W sensownie nowych dockerach (>1.13) `tiny` jest domyślnie wstrzykiwane po podaniu flagi `--init` do komendy `docker container run`.

```
FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5 as build
RUN apt-get update
RUN apt-get install -y --no-install-recommends \
	      build-essential gcc

WORKDIR /usr/app
COPY requirements.txt .
RUN python -m venv env
ENV PATH=/usr/app/env/bin:$PATH
RUN pip install -r requirements.txt

FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5
RUN groupadd -g 1001 python && \
    useradd -r -u 1001 -g python python
USER 1001

WORKDIR /usr/app

ENV TINI_VERSION v0.19.0
ADD https://github.com/krallin/tini/releases/download/${TINI_VERSION}/tini /tini
ADD https://github.com/krallin/tini/releases/download/${TINI_VERSION}/tini.asc /tini.asc
RUN gpg --batch --keyserver hkp://keyserver.ubuntu.com:80 --recv-keys 595E85A6B1B4779EA4DAAEC70B588DFF0527A9B7 \
 && gpg --batch --verify /tini.asc /tini
RUN chmod +x /tini

COPY --from=build /usr/app/env ./env
ENV PATH=/usr/app/env/bin:$PATH

COPY . .
ENTRYPOINT ["tini", "--", "python", "app.py" ]
```

### Serwer WSGI
Nawet jeśli używamy frameworka webowego takiego jak `flask` czy `django`, które od razu instalują nam minimalny serwer WSGI, w produkcyjnych obrazach dockerowych należy zamiast nich bezwzględnie stosować prawdziwą implmentację. Powodem jest to, że wbudowane w frameworki serwery udostępniają zbyt dużo opcji debugowania, które stają się łatwym do użycia wektorem ataków hackerskich. Dodatkowo nie są one przesadnie wydajne i nie mają wielu opcji kontrolujących zachowanie pełnoprawnych implementacji. W zależności czy używamy WSGI czy ASGI dobrymi wyborami są `Gunicorn`, `uWSGI` lub `uvicorn` w przypadku aplikacji asynchronicznych. Przykładowo dla Gunicorna nasz dockerfile powinien wyglądać następująco:

```
FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5 as build
RUN apt-get update
RUN apt-get install -y --no-install-recommends \
	      build-essential gcc

WORKDIR /usr/app
COPY requirements.txt .
RUN python -m venv env
ENV PATH=/usr/app/env/bin:$PATH
RUN pip install -r requirements.txt

FROM python:3.11.4-slim-bookworm@sha256:36b544be6e796eb5caa0bf1ab75a17d2e20211cad7f66f04f6f5c9eeda930ef5
RUN groupadd -g 1001 python && \
    useradd -r -u 1001 -g python python
USER 1001

WORKDIR /usr/app

ENV TINI_VERSION v0.19.0
ADD https://github.com/krallin/tini/releases/download/${TINI_VERSION}/tini /tini
ADD https://github.com/krallin/tini/releases/download/${TINI_VERSION}/tini.asc /tini.asc
RUN gpg --batch --keyserver hkp://keyserver.ubuntu.com:80 --recv-keys 595E85A6B1B4779EA4DAAEC70B588DFF0527A9B7 \
 && gpg --batch --verify /tini.asc /tini
RUN chmod +x /tini

COPY --from=build /usr/app/env ./env
ENV PATH=/usr/app/env/bin:$PATH

COPY . .
ENTRYPOINT ["tini", "--", "gunicorn", "--bind", "0.0.0.0:5000", "manage:app" ]
```

Sama instalacja gunicorna zwykle odbywa się przy użyciu `pip`a, więc wystarczy jak dodamy go do `requirements.txt`

### *Zadanie 1*
```
git checkout solution-8
git checkout -b my-solution-8
```
* [ ] Skonteneryzuj aplikację `dirwatcher`, korzystając ze wskazówek w notatkach:
  - wybierz właściwy obraz
  - zadbaj o cache'owanie
  - ogranicz uprawnienia
  - dobierz serwer http
  - zadbaj o poprawny init
  - zmierz rozmiar gotowego obrazu
